<h1>The Battle of Neighbourhoods (Week 2)</h1>

### Import necessary Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge beautifulsoup4 --yes

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# # Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

# #!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'FGHDQZF0EA3OHDQUDDU0YC0UKJG5UNISL55BC4H3SF0ZVIPQ' # your Foursquare ID
CLIENT_SECRET = 'HBRE2I2LJQAIVWLO35BTKQNRIH0YX43OMAN3I0QKFY5JCUQD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FGHDQZF0EA3OHDQUDDU0YC0UKJG5UNISL55BC4H3SF0ZVIPQ
CLIENT_SECRET:HBRE2I2LJQAIVWLO35BTKQNRIH0YX43OMAN3I0QKFY5JCUQD


### We obtain the data from 'https://es.wikipedia.org/wiki/Anexo:Distritos_de_Madrid', perform the necessary transformations and obtain the coordinate data for each neighborhood.

In [3]:
url = 'https://es.wikipedia.org/wiki/Anexo:Distritos_de_Madrid'
df = pd.read_html(url, header=0)[0]
df.rename(columns={'Nombre':'Name'}, inplace=True)  # Translate
df = df[['Name']] + ", Madrid"    
df['Latitude'] = ""
df['Longitude'] = ""
df = df.drop(len(df)-1)   # Erase the last row for totals

for i in df.index:
    address = df["Name"][i]
    geolocator = Nominatim(user_agent="mad_explorer")
    location = geolocator.geocode(address)  
    df['Latitude'][i] = location.latitude
    df['Longitude'][i] = location.longitude
    
df

,Name,Latitude,Longitude
0,"Centro, Madrid",40.4177,-3.70791
1,"Arganzuela, Madrid",40.3981,-3.69373
2,"Retiro, Madrid",40.4111,-3.67606
3,"Salamanca, Madrid",40.427,-3.6806
4,"Chamartín, Madrid",40.459,-3.67613
5,"Tetuán, Madrid",40.4606,-3.69828
6,"Chamberí, Madrid",40.4362,-3.70383
7,"Fuencarral-El Pardo, Madrid",40.5563,-3.77859
8,"Moncloa-Aravaca, Madrid",40.4395,-3.7442
9,"Latina, Madrid",40.4035,-3.73615


### We obtain the geographical location of the center of Madrid

In [4]:
address = 'Madrid'

geolocator = Nominatim(user_agent="mad_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid City are 40.4167047, -3.7035825.


### Create a map of Madrid with neighborhoods superimposed on top.

In [14]:
# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(df['Latitude'], df['Longitude'], df['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

### Let's get the top 100 venues that are in all the neighborhoods in Madrid within a radius of 500 meters.

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):          
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
madrid_venues = getNearbyVenues(names=df['Name'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude']
                               )
print(madrid_venues.shape)    # print shape

(371, 7)


In [8]:
madrid_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Arganzuela, Madrid",30,30,30,30,30,30
"Barajas, Madrid",30,30,30,30,30,30
"Carabanchel, Madrid",15,15,15,15,15,15
"Centro, Madrid",30,30,30,30,30,30
"Chamartín, Madrid",30,30,30,30,30,30
"Chamberí, Madrid",30,30,30,30,30,30
"Ciudad Lineal, Madrid",29,29,29,29,29,29
"Hortaleza, Madrid",20,20,20,20,20,20
"Latina, Madrid",8,8,8,8,8,8


### Analyze Each Neighborhood and group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [9]:
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
madrid_onehot['Neighbourhood'] = madrid_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

madrid_grouped = madrid_onehot.groupby('Neighbourhood').mean().reset_index()
madrid_grouped

,Neighbourhood,American Restaurant,Arcade,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Garden,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Café,Candy Store,Castle,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Dessert Shop,Diner,Dog Run,Donut Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Himalayan Restaurant,Historic Site,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Noodle House,Opera House,Park,Pastry Shop,Perfume Shop,Pharmacy,Pizza Place,Plaza,Pool,Pub,Ramen Restaurant,Resort,Restaurant,Rock Club,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Spanish Restaurant,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Tapas Restaurant,Theater,Thrift / Vintage Store,Trade School,Wine Bar,Wine Shop,Women's Store
0,"Arganzuela, Madrid",0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.033333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.066667,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.133333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
1,"Barajas, Madrid",0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.0,0.233333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.0000,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000
2,"Carabanchel, Madrid",0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.00

### Let's print each neighborhood along with the top 3 most common venues and put that into a pandas dataframe

In [10]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [11]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = madrid_grouped['Neighbourhood']

for ind in np.arange(madrid_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Arganzuela, Madrid",Tapas Restaurant,Grocery Store,Bakery
1,"Barajas, Madrid",Hotel,Spanish Restaurant,Argentinian Restaurant
2,"Carabanchel, Madrid",Tapas Restaurant,Restaurant,Spanish Restaurant
3,"Centro, Madrid",Plaza,Hostel,Bookstore
4,"Chamartín, Madrid",Restaurant,Spanish Restaurant,Supermarket
5,"Chamberí, Madrid",Tapas Restaurant,Bar,Theater
6,"Ciudad Lineal, Madrid",Spanish Restaurant,Restaurant,Gastropub
7,"Hortaleza, Madrid",Spanish Restaurant,Park,Supermarket
8,"Latina, Madrid",Grocery Store,Supermarket,Park
9,"Moncloa-Aravaca, Madrid",Castle,Bus Stop,Park


In [12]:
df_filtered = neighbourhoods_venues_sorted.filter(items=['Neighbourhood', '1st Most Common Venue'])
df_filtered = df_filtered.loc[(df_filtered['1st Most Common Venue'] == 'Spanish Restaurant')]
df_filtered.reset_index(inplace=True, drop = True)
df_filtered

,Neighbourhood,1st Most Common Venue
0,"Ciudad Lineal, Madrid",Spanish Restaurant
1,"Hortaleza, Madrid",Spanish Restaurant
2,"Retiro, Madrid",Spanish Restaurant
3,"Tetuán, Madrid",Spanish Restaurant


Once we have obtained the list of neighborhoods, we can recommend one of these to open a local with typical national food or, better said: "Spanish Restaurant". The reason for recommending opening a business of this type in neighborhoods with more of this type of business is that it can be very touristy or gastronomic areas and therefore a greater possibility of having more clients.
So we can recommend opening a "Spanish Restaurant" in the following neighborhoods:

In [13]:
df_filtered[['Neighbourhood']]

,Neighbourhood
0,"Ciudad Lineal, Madrid"
1,"Hortaleza, Madrid"
2,"Retiro, Madrid"
3,"Tetuán, Madrid"
